<h3>Segmenting Toronton Neighborhood - Part 1</h3>
<br>
<b>Scraping Wikipedia's page</b>

The below block reads the Wikipedia file into BeautifulSoup

In [1]:
import urllib.request
import pandas as pd 
  
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = urllib.request.urlopen(url)

from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

The below block find the postal code table in the page and:
<li>Ignore "Not assigned" Boroughs</li>
<li>Replace "Not assigned" Neighborhoods by the name of the Borough</li>
<li>Loads the pandas DataFrame with the data</li>

In [2]:
right_table=soup.find('table', class_='wikitable sortable')

A=[]
B=[]
C=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        zcode = cells[0].find(text=True).replace('\n','')
        bor = cells[1].find(text=True).replace('\n','')
        nbor = cells[2].find(text=True).replace('\n','')
        if bor != 'Not assigned':
            A.append(zcode)
            B.append(bor)
            if nbor != 'Not assigned':
                C.append(nbor)
            else:
                C.append(bor)
        
import pandas as pd
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
display(df)
df.shape

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


(103, 3)

<h3>Segmenting Toronton Neighborhood - Part 2</h3>
<br>
<b>Associating Coordinates</b>

The below block reads the CSV file and associate the coordinates with each postal code in the df DataFrame

In [6]:
df2 = pd.read_csv('https://cocl.us/Geospatial_data')
df2.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) 

df_inner = pd.merge(df, df2, on='PostalCode', how='inner')
display(df_inner)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
